Example below is to show how to parse J1939 CAN log in mdf formate to get J1939 DM01 DTC Diagnostics.

References:
* https://github.com/CSS-Electronics/api-examples/tree/master/examples/data-processing
* https://asammdf.readthedocs.io
* https://github.com/danielhrisca/asammdf


In [1]:
%load_ext autoreload
%autoreload 2

from etils import ecolab
from asammdf import MDF
from pathlib import Path

import pandas as pd
from J1939_PGN import J1939_PGN, J1939_PDU
import j1939
import cantools

from utils.mdf_css_proc_utils import  ProcessData, MultiFrameDecoder
import  utils.mdf_can_log_utils as utls

ldf is not supported
xls is not supported
xlsx is not supported


In [2]:
# input data
log_file1 = r"input\vehicle_A_can_log_1.mf4"
# J1939 DM01 is done based on Transport protocol, so for work we would need next PGN
target_msg_pgn={'DM01':65226, 'TPCM':60416,'TPDT':60160};

# They also could be retrieved by j1939.ParameterGroupNumber.PGN.DM01
[j1939.ParameterGroupNumber.PGN.DM01,
j1939.ParameterGroupNumber.PGN.TP_CM,
j1939.ParameterGroupNumber.PGN.DATATRANSFER];

{'DM01': 65226, 'TPCM': 60416, 'TPDT': 60160}

[65226, 60416, 60160]

In [3]:
# get mdf file and CAN trace in mdf format
mdf1 = MDF(log_file1)
mdf1_trace=utls.mdf_get_trace(mdf1)
mdf1_trace.shape;
mdf1_trace.columns;
mdf1_trace.index;
mdf1_trace.head();

(515834, 14)

Index(['TimeStamp', 'BusChannel', 'ID', 'IDE', 'Dir', 'Name', 'Event Type',
       'Details', 'ESI', 'EDL', 'BRS', 'DLC', 'DataLength', 'DataBytes'],
      dtype='object')

RangeIndex(start=0, stop=515834, step=1)

,TimeStamp,BusChannel,ID,IDE,Dir,Name,Event Type,Details,ESI,EDL,BRS,DLC,DataLength,DataBytes
0,0.001090,1,234291459,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[105, 76, 60, 39, 161, 64, 113, 6]"
1,0.001100,1,99877105,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[159, 237, 224, 6, 16, 0, 249, 50]"
2,0.001103,1,234098929,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[103, 124, 101, 124, 78, 135, 80, 135]"
3,0.001105,2,436166821,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[18, 0, 18, 0, 18, 0, 27, 0]"
4,0.001108,8,769,0,,,CAN Frame,,No error,Standard CAN,0,8,8,"[0, 144, 2, 58, 98, 0, 0, 1]"


In [4]:
# update with j1939 pgn, sa and da
mdf1_trace=utls.can_trace_df_update_j1939_info(mdf1_trace)
mdf1_trace.shape;
mdf1_trace.columns;
mdf1_trace.index;
mdf1_trace.head(3);

(515834, 18)

Index(['ID', 'TimeStamp', 'BusChannel', 'IDE', 'Dir', 'Name', 'Event Type',
       'Details', 'ESI', 'EDL', 'BRS', 'DLC', 'DataLength', 'DataBytes',
       'msg_pdu', 'msg_pgn', 'msg_sa', 'msg_da'],
      dtype='object')

RangeIndex(start=0, stop=515834, step=1)

,ID,TimeStamp,BusChannel,IDE,Dir,Name,Event Type,Details,ESI,EDL,BRS,DLC,DataLength,DataBytes,msg_pdu,msg_pgn,msg_sa,msg_da
0,234291459,0.001090,1,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[105, 76, 60, 39, 161, 64, 113, 6]",PDU2,128769.0,3.0,None
1,99877105,0.001100,1,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[159, 237, 224, 6, 16, 0, 249, 50]",PDU2,128000.0,241.0,None
2,234098929,0.001103,1,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[103, 124, 101, 124, 78, 135, 80, 135]",PDU2,128016.0,241.0,None


In [5]:
# filter on only interesting messages 
mdf1_trace2=mdf1_trace[mdf1_trace['msg_pgn'].isin(target_msg_pgn.values())]
mdf1_trace2.shape
mdf1_trace2.head(3)

,ID,TimeStamp,BusChannel,IDE,Dir,Name,Event Type,Details,ESI,EDL,BRS,DLC,DataLength,DataBytes,msg_pdu,msg_pgn,msg_sa,msg_da
50,419351131,0.005992,1,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[4, 255, 214, 226, 240, 1, 255, 255]",PDU2,65226.0,91.0,None
969,419351202,0.119021,2,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[4, 255, 128, 26, 208, 1, 255, 255]",PDU2,65226.0,162.0,None
1515,419351207,0.178968,2,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[4, 255, 128, 26, 208, 1, 255, 255]",PDU2,65226.0,167.0,None


In [6]:
# check for DM01 message
dm01_df0=mdf1_trace2[(mdf1_trace2.msg_sa==90) & (mdf1_trace2.msg_pgn==target_msg_pgn['DM01'])]
dm01_df0.shape;
dm01_df0.head(3);


(0, 18)

,ID,TimeStamp,BusChannel,IDE,Dir,Name,Event Type,Details,ESI,EDL,BRS,DLC,DataLength,DataBytes,msg_pdu,msg_pgn,msg_sa,msg_da


We see there's no DM01 messages in CAN trace - because they are represented in transport protocol. Lets try to get it. 

Its messages could be seen as TPCM and TPDT frames, that compose together DM01 messages. See example below - DM01 is not presented in original CAN trace but created by Vector Canalyzer. We will do the same using python utils next
![J1939_DM01_TP_example.png](./attachment/J1939_DM01_TP_example.png)

In [7]:
#  Now lets try to do the same - combine multi frames and get DM01
tp = MultiFrameDecoder("j1939")
mdf1_tr_cb = tp.combine_tp_frames(mdf1_trace2)
mdf1_tr_cb=utls.can_trace_df_update_j1939_info(mdf1_tr_cb)
mdf1_tr_cb.shape;
mdf1_tr_cb.head(3);

(1721, 18)

,ID,BusChannel,IDE,Dir,Name,Event Type,Details,ESI,EDL,BRS,DLC,DataLength,DataBytes,msg_pdu,msg_pgn,msg_sa,msg_da,SA
TimeStamp,,,,,,,,,,,,,,,,,,
0.005992,419351131,1,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[4, 255, 214, 226, 240, 1, 255, 255]",PDU2,65226,91,None,NaN
0.119021,419351202,2,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[4, 255, 128, 26, 208, 1, 255, 255]",PDU2,65226,162,None,NaN
0.178968,419351207,2,1,,,CAN Frame,,No error,Standard CAN,0,8,8,"[4, 255, 128, 26, 208, 1, 255, 255]",PDU2,65226,167,None,NaN


Now CAN trace DF is updated with all possible DM01 messages, lets try to see it from source address SA=90

In [8]:
# show all DM01 from SA=90
dm01_df1=mdf1_tr_cb[(mdf1_tr_cb.msg_sa==90) & (mdf1_tr_cb.msg_pgn==target_msg_pgn['DM01'])]
dm01_df1.shape;
dm01_df1.head();

(60, 18)

,ID,BusChannel,IDE,Dir,Name,Event Type,Details,ESI,EDL,BRS,DLC,DataLength,DataBytes,msg_pdu,msg_pgn,msg_sa,msg_da,SA
TimeStamp,,,,,,,,,,,,,,,,,,
0.339087,419351130,2,1,,,CAN Frame,,No error,Standard CAN,0,0,63,"[16, 255, 133, 232, 226, 1, 200, 231, 228, 1, ...",PDU2,65226,90,None,90.0
1.339342,419351130,2,1,,,CAN Frame,,No error,Standard CAN,0,0,63,"[16, 255, 133, 232, 226, 1, 200, 231, 228, 1, ...",PDU2,65226,90,None,90.0
2.338152,419351130,2,1,,,CAN Frame,,No error,Standard CAN,0,0,63,"[16, 255, 133, 232, 226, 1, 200, 231, 228, 1, ...",PDU2,65226,90,None,90.0
3.339117,419351130,2,1,,,CAN Frame,,No error,Standard CAN,0,0,63,"[16, 255, 133, 232, 226, 1, 200, 231, 228, 1, ...",PDU2,65226,90,None,90.0
4.339186,419351130,2,1,,,CAN Frame,,No error,Standard CAN,0,0,63,"[16, 255, 133, 232, 226, 1, 200, 231, 228, 1, ...",PDU2,65226,90,None,90.0


Now go next - try to understand whats these DM01 messages contains, which diagnostics we could get from there

In [9]:
# try to parse payload data using j1939 dbc
dbc_file_path = r"dbc\j1939_DM01.dbc"
db = cantools.database.load_file(dbc_file_path)
DM01_CAN_message = db.get_message_by_name('DM01')

In [10]:
# take bytes from the 1st row and decode it with dbc
info=dm01_df1['DataBytes'].iloc[0]
info=bytes(info);
DM01_decoded_signals = DM01_CAN_message.decode(info);

b'\x10\xff\x85\xe8\xe2\x01\xc8\xe7\xe4\x01\xc7\x19\x0b\x01\x82\x03\x15\x01\x80\x1a\xd0\x01\xe8\xec\xf0\x01e\xe9\xed\x01\xb8\xe9\xed\x01\x8386\x01\xd6\xe7\xec\x01\xd6\xe7\xe2\x01\xd6\xe7\xe0\x01\x82\x03\x02\x01%\xea\xed\x01\xff\xff\xff\xff\xff'

{'PLStatus': 'Lamp Off',
 'AWLStatus': 'Lamp Off',
 'RSLState': 'Lamp On',
 'MILStatus': 'Lamp Off',
 'FlashProtectLamp': 'Unavailable / Do Not Flash',
 'FlashAmberWarningLamp': 'Unavailable / Do Not Flash',
 'FlashRedStopLamp': 'Unavailable / Do Not Flash',
 'FlashMalfuncIndicatorLamp': 'Unavailable / Do Not Flash',
 'DTC1': 31647877,
 'DTC2': 31778760,
 'DTC3': 17504711,
 'DTC4': 18154370,
 'DTC5': 30415488}

Validate - check against results from Vector Canalyzer.
We could see the DTC numbers looks correct - see 31647877. 31778760 and etc
![canlog_38837_DM01.png](./attachment/canlog_38837_DM01.png)

Lets see original data from the trace using Vector Canalyzer data 

In [11]:
# to validate lets import parsed data from vector canalyzer - here's a table with DM01.DTC only (yet not SPN-FMI)
vector_file=r'input/vector_canalyzer_export_trace_DM01_vehicle_A_can_log_1.csv'
dm01_dtc_vector_df=pd.read_csv(vector_file)
dm01_dtc_vector_df.head()

,Time[s],j1939::DM01::MILStatus,j1939::DM01::AWLStatus,j1939::DM01::RSLState,j1939::DM01::PLStatus,j1939::DM01::DTC5,j1939::DM01::DTC4,j1939::DM01::DTC3,j1939::DM01::DTC2,j1939::DM01::DTC1
0,0.833994,0,0,1,0,30415488,18154370,17504711,31778760,31647877
1,1.833996,0,0,1,0,30415488,18154370,17504711,31778760,31647877
2,2.834154,0,0,1,0,30415488,18154370,17504711,31778760,31647877
3,3.834005,0,0,1,0,30415488,18154370,17504711,31778760,31647877
4,4.834139,0,0,1,0,30415488,18154370,17504711,31778760,31647877


or using diagnostics tools - data already parsed into SPN FMI
![vector_canalyzer_parse_DM01_DTC_SPN_FMI.png](./attachment/vector_canalyzer_parse_DM01_DTC_SPN_FMI.png)

In [12]:
# the same data from Vector table above in excel format
vector_file=r'input/vector_canalyzer_export_DTC_vehicle_A_can_log_1.csv'
dm01_dtc_spn_vector_df=pd.read_csv(vector_file)
dm01_dtc_spn_vector_df.head()

,Time,Node,SPN,FMI
0,0.833994,DCU,518277,2
1,0.833994,DCU,518088,4
2,0.833994,DCU,6599,11
3,0.833994,DCU,898,21
4,0.833994,DCU,400000,16


Now lets try to do the same using Python tools

In [13]:
#  decode DTC into SPN FMI
# using custom functions
DTC0=utls.decode_dtc(DM01_decoded_signals['DTC1']);
# or using J1939 library
DTC1=j1939.DTC(DM01_decoded_signals['DTC1'])
DTC1.__dict__;

{'dtc': 31647877, 'spn': 518277, 'fmi': 2, 'oc': 1, 'cm': 0}

{'_dtc': 31647877, '_spn': 518277, '_fmi': 2, '_oc': 1, '_cm': 0}

Looks the same as from Vector Canalyzer.

Next - try to parse all data from log in table format

In [14]:
#  now process all dtc
DTCs=['DTC1','DTC2','DTC3','DTC4','DTC5']
dm01_df1=mdf1_tr_cb[(mdf1_tr_cb.msg_pgn==target_msg_pgn['DM01'])]
cols2add=['BusChannel','msg_sa']
L=[]
for idx, row in dm01_df1.iterrows():
    info=row['DataBytes']
    info=bytes(info)
    DM01_decoded_signals = DM01_CAN_message.decode(info,allow_truncated=True)
    for DTC_name in DTCs:
        if DTC_name not in DM01_decoded_signals:
            continue
        DTC=utls.decode_dtc(DM01_decoded_signals[DTC_name])
        DTC_row=row[cols2add]
        DTC_row['time']=idx
        DTC_row['SPN']=DTC['spn']
        DTC_row['FMI']=DTC['fmi']
        L.append(DTC_row) 
         
DTC_df=pd.DataFrame(L)
DTC_df.reset_index(inplace=True)
DTC_df.head();

,index,BusChannel,msg_sa,time,SPN,FMI
0,0.005992,1,91,0.005992,516822,16
1,0.119021,2,162,0.119021,400000,16
2,0.178968,2,167,0.178968,400000,16
3,0.195964,2,166,0.195964,400000,16
4,0.195964,2,166,0.195964,510031,2


Now we have big table with all SPN FMI as separate row.
Next combine it with known source and SPN names

In [15]:
DTC_db_df=pd.read_csv(r'input\DTC_SPN_FMI_db.csv')
ECU_db_df=pd.read_csv(r'input\ECU_NAMES.csv')

In [16]:
#  extend spn table with source address
DTC_db_df=DTC_db_df.merge(ECU_db_df,how='left',left_on='source',right_on='node_name')
DTC_db_df.head();

,SPN name,SPN,FMI,source,node_name,node_id_dec
0,DC-DC Converter MalfunctionFault Code 30,519320,1,CABIN_IO_1_ECU,CABIN_IO_1_ECU,163
1,DC-DC Converter MalfunctionFault Code 30,519320,0,CABIN_IO_1_ECU,CABIN_IO_1_ECU,163
2,Regenerative Braking ErrorFault Code 28,519326,1,CABIN_IO_1_ECU,CABIN_IO_1_ECU,163
3,Regenerative Braking ErrorFault Code 28,519326,0,CABIN_IO_1_ECU,CABIN_IO_1_ECU,163
4,Battery State of Charge ErrorFault Code 8,519325,1,CABIN_IO_1_ECU,CABIN_IO_1_ECU,163


In [17]:
#  extend DTC table from can trace with spn names
DTC_df=DTC_df.merge(DTC_db_df,how='left',left_on=['msg_sa','SPN','FMI'],right_on=['node_id_dec','SPN','FMI'])
DTC_df.head();

,index,BusChannel,msg_sa,time,SPN,FMI,SPN name,source,node_name,node_id_dec
0,0.005992,1,91,0.005992,516822,16,NaN,NaN,NaN,NaN
1,0.119021,2,162,0.119021,400000,16,Isolation Fault DetectedFault Code 5,FRONT_IO_4_ECU,FRONT_IO_4_ECU,162.0
2,0.178968,2,167,0.178968,400000,16,Isolation Fault DetectedFault Code 5,REAR_IO_2_ECU,REAR_IO_2_ECU,167.0
3,0.195964,2,166,0.195964,400000,16,Isolation Fault DetectedFault Code 5,REAR_IO_1_ECU,REAR_IO_1_ECU,166.0
4,0.195964,2,166,0.195964,510031,2,Drive Motor OverloadFault Code 17,REAR_IO_1_ECU,REAR_IO_1_ECU,166.0
